In [60]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression # Linear Regression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import BalancedRandomForestClassifier # RandomForestClassifier

In [62]:
%matplotlib inline
from sklearn.datasets import make_blobs

In [63]:
# importing test weather data
weather_df = pd.read_csv("SevereWeatherDetails_appended.csv", low_memory=False)

In [64]:
weather_df.head()

,EVENT_ID,STATE,STATE_FIPS,YEAR,MONTH_NAME,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,MAGNITUDE,CATEGORY,TOR_F_SCALE,TOR_LENGTH,TOR_WIDTH,BEGIN_LAT,BEGIN_LON
0,10096222,OKLAHOMA,40.0,1950,April,Tornado,0,0,0,0,250K,0,0.0,NaN,F3,3.4,400.0,35.12,-99.20
1,10120412,TEXAS,48.0,1950,April,Tornado,0,0,0,0,25K,0,0.0,NaN,F1,11.5,200.0,31.90,-98.60
2,10104927,PENNSYLVANIA,42.0,1950,July,Tornado,2,0,0,0,25K,0,0.0,NaN,F2,12.9,33.0,40.58,-75.70
3,10104928,PENNSYLVANIA,42.0,1950,July,Tornado,0,0,0,0,2.5K,0,0.0,NaN,F2,0.0,13.0,40.60,-76.75
4,10104929,PENNSYLVANIA,42.0,1950,July,Tornado,0,0,0,0,2.5K,0,0.0,NaN,F0,0.0,33.0,41.63,-79.68


In [65]:
# dropping columns not needed
weather_df = weather_df.drop(columns=['STATE','STATE_FIPS','MONTH_NAME','INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT',
                                     'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'MAGNITUDE', 'TOR_F_SCALE', 'TOR_LENGTH',
                                     'TOR_WIDTH', 'BEGIN_LAT', 'BEGIN_LON','CATEGORY', 'EVENT_ID'])

In [66]:
weather_df.head()

,YEAR,EVENT_TYPE
0,1950,Tornado
1,1950,Tornado
2,1950,Tornado
3,1950,Tornado
4,1950,Tornado


In [67]:
weather_year = weather_df.groupby(['YEAR']).count()

In [68]:
weather_year.head()

,EVENT_TYPE
YEAR,
1950,223
1951,269
1952,272
1953,492
1954,609


In [69]:
carbon_df = pd.read_csv("co2_byYear.csv", low_memory=False)

In [70]:
carbon_df

,year,co2,co2_per_capita,co2_per_unit_energy,coal_co2,cement_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,...,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,population,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp
0,1950,23193.336,943.286,0.000,15959.768,272.887,223.550,1110.931,5611.231,9.976,...,0.000,0.00,0.000,0.00,0.000,7.570713e+09,1.585967e+13,0.000,0.000,0.000
1,1951,24765.194,1214.978,0.000,16981.583,309.265,243.305,1306.674,5907.881,11.002,...,0.000,0.00,0.000,0.00,0.000,7.712655e+09,7.992201e+12,0.000,0.000,0.000
2,1952,25203.424,834.737,0.000,16927.201,331.430,259.877,1405.429,6263.420,10.710,...,0.000,0.00,0.000,0.00,0.000,7.852545e+09,8.401431e+12,0.000,0.000,0.000
3,1953,25929.893,880.578,0.000,17104.163,368.454,248.979,1498.933,6689.993,12.910,...,0.000,0.00,0.000,0.00,0.000,7.992062e+09,8.884521e+12,0.000,0.000,0.000
4,1954,26744.552,1378.627,0.000,17319.801,400.182,236.584,1588.220,7182.609,11.442,...,0.000,0.00,0.000,0.00,0.000,8.133217e+09,9.099744e+12,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,2016,124705.642,1156.831,44.924,48325.275,4990.527,1438.122,27011.160,41892.058,1048.509,...,1415.378,17424.08,345.519,6291.93,105.005,2.239131e+10,2.147076e+14,366170.181,5562418.068,208.589
67,2017,126304.716,1156.251,18.863,48587.789,5067.569,1576.265,27603.881,42412.614,1056.584,...,0.000,0.00,0.000,0.00,0.000,2.264281e+10,2.211738e+14,366923.317,3678183.705,0.000
68,2018,128452.538,1152.783,18.444,49368.748,5286.347,1531.205,28781.873,42422.168,1062.207,...,0.000,0.00,0.000,0.00,0.000,2.289184e+10,2.276454e+14,376507.961,3698476.648,0.000
69,2019,128422.887,1165.000,18.142,48731.161,5408.677,1611.674,29061.768,42547.578,1062.016,...,0.000,0.00,0.000,0.00,0.000,2.313895e+10,0.000000e+00,380148.487,3659567.475,0.000


In [71]:
# dropping columns not needed
carbon_df = carbon_df.drop(columns=['co2_per_capita','co2_per_unit_energy', 'coal_co2', 'cement_co2', 'flaring_co2', 'gas_co2',
                                         'oil_co2', 'other_industry_co2', 'ghg_per_capita', 'methane', 'methane_per_capita',
                                         'nitrous_oxide', 'nitrous_oxide_per_capita', 'population', 'gdp', 'primary_energy_consumption',
                                         'energy_per_capita', 'energy_per_gdp', 'total_ghg'])

In [72]:
carbon_df

,year,co2
0,1950,23193.336
1,1951,24765.194
2,1952,25203.424
3,1953,25929.893
4,1954,26744.552
...,...,...
66,2016,124705.642
67,2017,126304.716
68,2018,128452.538
69,2019,128422.887


In [73]:
year_df = weather_year.merge(carbon_df, left_on='YEAR', right_on='year')

In [74]:
year_df.head()

,EVENT_TYPE,year,co2
0,223,1950,23193.336
1,269,1951,24765.194
2,272,1952,25203.424
3,492,1953,25929.893
4,609,1954,26744.552


In [75]:
# Use numpy to convert to arrays
import numpy as np
# Labels are the values we want to predict
labels = np.array(year_df['EVENT_TYPE'])
# Remove the labels from the features
# axis 1 refers to the columns
features= year_df.drop('EVENT_TYPE', axis = 1)
# Saving feature names for later use
feature_list = list(year_df.columns)
# Convert to numpy array
features = np.array(year_df)

In [76]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [77]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (53, 3)
Training Labels Shape: (53,)
Testing Features Shape: (18, 3)
Testing Labels Shape: (18,)


In [78]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('EVENT_TYPE')]
# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  0.0


In [79]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 500, random_state = 40)
# Train the model on training data
rf.fit(train_features, train_labels);

In [80]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 1542.6 degrees.


In [81]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 87.94 %.
